In [ ]:
%matplotlib inline
import numpy as np
import cv2
import os
from tqdm import tqdm
import imutils
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D , BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.morphology import watershed as skwater
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D , BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### DATA NO-NO

In [ ]:
path1 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_NO_NO/INME_YOK'
path2 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_NO_NO/ISKEMI'
path3 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_NO_NO/KANAMA'

### DATA YES-NO

In [ ]:
"""
path1 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_YES_NO/INME_YOK'
path2 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_YES_NO/ISKEMI'
path3 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_YES_NO/KANAMA'
"""

"\npath1 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_YES_NO/INME_YOK'\npath2 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_YES_NO/ISKEMI'\npath3 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_YES_NO/KANAMA'\n"

### DATA YES-YES

In [ ]:
"""
path1 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_YES_YES/INME_YOK'
path2 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_YES_YES/ISKEMI'
path3 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_YES_YES/KANAMA'
"""

In [ ]:
image_list = []
label_list=[]
def loaddata(path, label):
    all_images = os.listdir(path)
    for image in tqdm(all_images):
        img = cv2.imread(os.path.join(path, image))      
        image_list.append(img)
        label_list.append(label)    

In [ ]:
loaddata(path1,0)
loaddata(path2,1)
loaddata(path3,1)

100%|██████████| 1093/1093 [00:03<00:00, 303.59it/s]


In [ ]:
img = np.array(image_list)
lbl = np.array(label_list)

In [ ]:
img[0][44]

In [ ]:
del image_list
del label_list
import gc
gc.collect()

36646

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(img, lbl, test_size=0.1, random_state=32, shuffle= True) 

In [ ]:
del img
del lbl
import gc
gc.collect()

15560

# INCEPTION MODEL

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, decode_predictions, preprocess_input

Inception_model = InceptionV3(include_top=False, pooling='max', weights="imagenet")


In [ ]:
for layer in Inception_model.layers:
    layer.trainable = False
Inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_998 (Conv2D)             (None, None, None, 3 864         input_12[0][0]                   
__________________________________________________________________________________________________
batch_normalization_997 (BatchN (None, None, None, 3 96          conv2d_998[0][0]                 
__________________________________________________________________________________________________
activation_997 (Activation)     (None, None, None, 3 0           batch_normalization_997[0][0]    
_______________________________________________________________________________________

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
    
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
modelIncep = Sequential()
modelIncep.add(Inception_model)
modelIncep.add(Flatten())
modelIncep.add(Dense(1024, activation='relu'))
modelIncep.add(Dropout(0.2))
modelIncep.add(Dense(512, activation='relu'))
modelIncep.add(Dropout(0.2))
modelIncep.add(Dense(256, activation='relu'))
modelIncep.add(Dropout(0.2))
modelIncep.add(Dense(128, activation='relu'))
modelIncep.add(Dropout(0.2))
modelIncep.add(Dense(64, activation='relu'))
modelIncep.add(Dropout(0.2))
modelIncep.add(Dense(32, activation='relu'))
modelIncep.add(Dropout(0.2))
modelIncep.add(Dense(1, activation='sigmoid'))
modelIncep.layers[0].trainable = False

"""
modelIncep.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',
                                                                        tf.keras.metrics.Precision(),
                                                                        tf.keras.metrics.Recall(),
                                                                        f1_m])
"""
modelIncep.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
modelIncep.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
flatten_9 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_63 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_54 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_64 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_55 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_65 (Dense)             (None, 256)              

In [ ]:
history = modelIncep.fit(X_train,y_train,validation_split=0.1,batch_size=10, epochs=50)

Epoch 1/50
539/539 [==============================] - 26s 40ms/step - loss: 3.6549 - accuracy: 0.5956 - val_loss: 0.6695 - val_accuracy: 0.6728
Epoch 2/50
539/539 [==============================] - 21s 39ms/step - loss: 0.6509 - accuracy: 0.6556 - val_loss: 0.6328 - val_accuracy: 0.6728
Epoch 3/50
539/539 [==============================] - 21s 38ms/step - loss: 0.6415 - accuracy: 0.6625 - val_loss: 0.6324 - val_accuracy: 0.6728
Epoch 4/50
535/539 [============================>.] - ETA: 0s - loss: 0.6407 - accuracy: 0.6634

KeyboardInterrupt: ignored

In [ ]:
scores = modelIncep.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
#PATH DEĞİŞTİR HER EĞİTİMDEN SONRA !!!!!!!!!!!!!!!!!!!!
modelIncep.save('brain_tumor_vgg16.h5') 

#  LOAD MODEL
#model1 = load_model('brain_tumor_vgg16.h5')

In [ ]:
ynew = modelIncep.predict_classes(X_test)
count = 0
index = 0
for img in X_test[100:150]:
    plt.figure()
    plt.title(ynew[index])
    plt.xlabel(y_test[index])
    plt.imshow(img)
    index = index +1